# The Energy Matrix & Education

### In this notebook we will clean and prepare the data for our Tableau Visualization. See below the datasets that we will use:

- [Energy consumption by source, country and year](https://github.com/owid/energy-data).
- [Education metrics & total population by country and year](http://hdr.undp.org/en/data).
- [Country attributes to allow for aggregation (e.g. by continent)](https://github.com/datasets/country-codes).

The energy data is already structured in a way close to what we need(courtesy of Our World in Data), while the education metrics are fragmented, so the latter will require more work.

## Energy Data

#### This Dataset has more data than we need. There are 3 steps we will take here to prepare the data for further use

- Remove unwanted fields. We want to keep the dimensions (Country, Country Code and Year) and the measures associated with the total consumption by a given country in a given year
- There are rows that holds data for countries aggregates, such as "South America". If we keep it, we will end up duplicating the consumption amounts for south american countries when aggregating
- We will also rename the measures related to each energy source now, as it is more convinient to do so before unpivoting the data

In [8]:
import pandas as pd
import copy

# Keeping original dataset with another name as we will fetch some values from this later on
df_energy_import = pd.read_csv('../Datasets\energy_data.csv')

### First, we will slice the dataframe column-wise, so we keep only the fields that we will use

In [9]:
# As the dataset has a lot of columns, it is good to start finding the 
# fields we want to keep by a common word in their names, if possible
dim_cols = ['iso_code', 'country', 'year']
measure_cols = [col for col in df_energy_import.columns if 'share_energy' in col]
cols_to_keep = copy.copy(dim_cols)
cols_to_keep.extend(measure_cols)

# Dropping unwanted fields
df_energy = df_energy_import[cols_to_keep]

df_energy.head()


,iso_code,country,year,biofuel_share_energy,coal_share_energy,fossil_share_energy,gas_share_energy,hydro_share_energy,low_carbon_share_energy,nuclear_share_energy,oil_share_energy,other_renewables_share_energy,renewables_share_energy,solar_share_energy,wind_share_energy
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now, we will slice it based on row values

In [10]:
# As we only have education data from 1990 onwards, we will filter this accordingly
# df_energy = df_energy[df_energy['year'] >= 1990]

# As we have many aggregates of countries
# We need to drop them as to not duplicate data & keep the analysis in terms of countries
entities_to_drop = [
    'World', 'Africa', 'Asia Pacific',
    'CIS', 'Central America', 'Eastern Africa',
    'Europe', 'Europe (other)', 'Middle Africa',
    'Middle East', 'Netherlands Antilles', 'North America',
    'OPEC', 'Other Asia & Pacific', 'Other CIS', 
    'Other Caribbean', 'Other Middle East', 'Other Northern Africa',
    'Other South America', 'Other Southern Africa', 'South & Central America',
    'South Africa', 'Western Africa', 'Western Sahara'
    ]

df_energy = df_energy[~df_energy['country'].isin(entities_to_drop)]



### To build the visualizations that we are aiming to in Tableau, we will need this data in long format, so we will unpivot it below

In [11]:
# We will rename the fields to how we want them to be displayed in the report view
# If we intended to keep the measures as fields, we could rename them in Tableau, but as
# we are unpivoting this (putting measure fields in one attribute field) it is better to do it now
measure_cols = [
    'Biofuels',
    'Coal',
    'Fossil Fuels',
    'Natural Gas',
    'Hydro',
    'Low Carbon',
    'Nuclear',
    'Oil',
    'Other Renewables',
    'Renewables',
    'Solar',
    'Wind'
    ]

renamed_cols =  copy.copy(dim_cols)
renamed_cols.extend(measure_cols)
df_energy.columns = renamed_cols

# In order to be able to measure correlation in Tableau, we will need unpivoted data
df_energy = df_energy.melt(
    id_vars=dim_cols, value_vars=measure_cols,
    var_name='energy_source', value_name='share'
)

df_energy = df_energy[~df_energy['share'].isna()].reset_index(drop=True)

### Finally, we will bring in two additional measures from the original original dataset and countries attributes from dim country table

In [12]:
df_energy_glb_metrics = df_energy_import[['iso_code', 'year', 'primary_energy_consumption', 'population']]
df_energy = df_energy.merge(df_energy_glb_metrics, on=['iso_code', 'year'], how='left')

df_countries = pd.read_csv('../Datasets\dim_country.csv')
df_countries = df_countries[[
    'ISO3166-1-Alpha-3', 'CLDR display name', 'Continent',
    'Region Name', 'Sub-region Name', 'Developed / Developing Countries'
    ]]

df_countries.columns = ['iso_code', 'country', 'continent', 'region name',
       'sub-region_name', 'developed_developing']

df_energy = df_energy.merge(df_countries, on='iso_code', how='left', suffixes=('', '_drop'))

df_energy.drop(axis=1, columns='country_drop', inplace=True)


In [13]:
df_energy.to_csv('..\Datasets/energy_date_long.csv')

## Education Data


Here we will need some more complex data cleaning & transformation.

Some countries names differ from these datasets and the country attribute table that we are using. This is not an issue for the energy dataset as it contains countries iso codes.

For that reason, I've created a mapping table to match the education datasets to the country dim table.
We will leverage that here to filter out rows that contains data for countries aggregates instead of doing by hand (as was done in the energy dataset) considering we already have a list of all countries contained in this datasets.

Additionally, it comes with bank columns between each year column. We are filtering that out by specifying the fields we wanna keep on importing

In [83]:

# We will use a list of all unique contry names contained in the UN education datasets
# to filter out unwanted rows
df_country_map = pd.read_csv('..\Datasets/un_to_iso_mapping.csv', encoding='latin 1')
un_countries_list = df_country_map['UN_Country'][~df_country_map['UN_Country'].isna()]
un_countries_list

# We are droping 2019 data as there are too many null values in the source for this year
years_cols = ['1990', '1995', '2000', '2005']
years_cols.extend([str(year) for year in range(2010, 2019)])
cols_to_keep = ['Country']
cols_to_keep.extend(years_cols)

# We are going to read all the education files and put them in the same dataframe
df_edu = pd.DataFrame(columns=['country', 'year', 'value', 'metric'])

# For that, we will need the file names
import os
edu_file_names = os.listdir('../Datasets\education_data_fragmented')
edu_folder_path = r'../Datasets\education_data_fragmented\\'

for file_name in edu_file_names:

    file_path = edu_folder_path + file_name

    df_temp = pd.read_csv(
        file_path, na_values='..', skiprows = 6,
        usecols=cols_to_keep, encoding='latin 1'
        )
    
    df_temp['Country'] = df_temp['Country'].str.strip()
    df_temp = df_temp[df_temp['Country'].isin(un_countries_list)]

    df_temp = df_temp.melt(id_vars='Country', value_vars=df_temp.columns[1:], var_name='year', value_name='value')
    df_temp.columns = ['country', 'year', 'value'] # Keeping capitalization consistent
    df_temp['metric'] = file_name[:-4] # dropping '.csv' from the name
    df_edu = df_edu.append(df_temp, ignore_index=True)


df_edu = df_edu.merge(right=df_country_map[['iso_code', 'UN_Country']], left_on='country', right_on='UN_Country', how='left')
df_edu.drop(axis=1, columns=['country', 'UN_Country'], inplace=True)
df_edu = df_edu[~df_edu['value'].isna()].reset_index(drop=True)
df_edu['year'] = df_edu['year'].astype('int')

In [84]:
df_edu.to_csv('../Datasets\education_data.csv', index=False)

### To avoid performance hits in Tableau when building the correlation Matrix, we are already joining both datasets here, keeping only country+year combinations that are present in both datasets

In [86]:
df_correlation = df_energy.merge(df_edu, on=['iso_code', 'year'], how='inner', suffixes=('', '_drop'))
df_correlation.to_csv('../Datasets\correlation_dataset.csv')